# 数据加载

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
pn.extension()

sys.path.append(os.path.abspath('../'))
os.environ["NUMBA_CUDA_DRIVER"] = "C:\\Windows\\System32\\nvcuda.dll"
from util.data_utils import load_csv_folder
from util.viz_utils import line_chart

In [ ]:
# 加载数据
data_dir = '../../data/sat1'  # 路径可根据实际情况调整
dataframes = load_csv_folder(data_dir)
file = list(dataframes.keys())[0]  # 选择第一个文件演示
df = dataframes[file]
print(f'加载数据文件: {file}, 行数: {len(df)}')
df.head()

# 实验

In [23]:
# 分箱算法
def group_by_bins(data, bins):
    """
    将数据按箱数进行分组
    
    参数:
        data: DataFrame 或类数组数据
        bins: 总箱数，每箱约4个点
    
    返回:
        分箱后的数据
    """
    output=data.copy()

    # 确定时间范围
    tmin = data.index.min()
    tmax = data.index.max()
    delta = tmax - tmin
    
    # 将时间戳映射到整数箱坐标
    output['group_idx'] = ((data.index - tmin) / delta * (bins - 1)).round().astype(int)

    return output

# def group_by_timedelta(data, timedelta):
#     output=data.copy()
#     return output

def group_by_time(data, freq='min'):
    """
    将数据按时间频率进行分组
    
    参数:
        data: DataFrame 或类数组数据
        freq: 时间频率，可选值为 'min'、'h'、'd' 等

    返回:
        分组后的数据
    """
    output=data.copy()
    output['group_idx'] = pd.Series(data.index).dt.floor(freq).values

    return output



In [27]:
def m4(data):
    """
    M4 算法实现，用于可视化保留的方式减少时间序列数据点数
    
    参数:
        data: 分组后数据，要求每组多于4个点
    返回:
        减采样后的数据
    """
    # 对每个箱计算极值
    result_indices = []
    value_col = data.columns[0]
    
    for bin_idx, group in data.groupby('group_idx'):
        # print(group)
        if len(group) > 0:
            # 找到时间最小/最大和值最小/最大的行索引
            tmin_idx = group.index.min()
            tmax_idx = group.index.max()
            vmin_idx = group[value_col].idxmin()
            vmax_idx = group[value_col].idxmax()
            
            # 将这些索引添加到结果中（可能有重复）
            result_indices.extend([tmin_idx, tmax_idx, vmin_idx, vmax_idx])
    
    # 去除重复索引
    # result_indices = list(set(result_indices))
    # print(result_indices)
    # 返回过滤后的数据
    result = data.loc[result_indices].drop(columns=['group_idx'])
    
    print(f"原始数据点数: {len(data)}, 处理后数据点数: {len(result)}")
    
    return result

def lttb(data):
    pass

freq = '5min'

downsampled = {}
for file in dataframes.keys():
    df = dataframes[file]
    grouped=group_by_time(df,freq)
    downsampled[file] = m4(grouped)

原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128810, 处理后数据点数: 419024
原始数据点数: 2128963, 处理后数据点数: 419024
原始数据点数: 2128815, 处理后数据点数: 419024
原始数据点数: 2128815, 处理后数据点数: 419024
原始数据点数: 2128963, 处理后数据点数: 419024


# 结果

In [35]:
df.index
raw=line_chart(dataframes)
# cooked=line_chart(downsampled)

AttributeError: 'str' object has no attribute 'min'